# Advanced RAG System Tutorial

This notebook provides practical examples and explanations for advanced features and customizations of our RAG system.

## 1. Experimenting with Different Document Types

Let's explore how the system handles different types of documents and their unique characteristics.

In [ ]:
from document_processor import DocumentProcessor
import os

# Initialize the processor with custom chunk sizes
processor = DocumentProcessor(chunk_size=500, chunk_overlap=100)

# Example: Process a technical document
tech_doc = """
Technical Specification Document
Version: 1.0

System Requirements:
- CPU: 2.0 GHz or higher
- RAM: 8GB minimum
- Storage: 20GB free space

Installation Steps:
1. Download the installer
2. Run setup.exe
3. Follow the wizard
4. Restart your computer

Configuration:
- Set memory limit to 4GB
- Enable auto-updates
- Configure backup schedule
"""

# Save to a temporary file
with open("tech_spec.txt", "w") as f:
    f.write(tech_doc)

# Process the document
documents = processor.process_file("tech_spec.txt")
print(f"Number of chunks: {len(documents)}")
print("\nFirst chunk:")
print(documents[0].page_content)

# Clean up
os.remove("tech_spec.txt")

## 2. Experimenting with Chunk Sizes

Different types of content may require different chunk sizes. Let's experiment with various configurations.

In [ ]:
def test_chunk_sizes(text, chunk_sizes):
    results = {}
    for size in chunk_sizes:
        processor = DocumentProcessor(chunk_size=size, chunk_overlap=size//5)
        documents = processor.process_documents([text])
        results[size] = {
            "num_chunks": len(documents),
            "avg_chunk_size": sum(len(d.page_content) for d in documents) / len(documents)
        }
    return results

# Test text
test_text = """
This is a test document to demonstrate how different chunk sizes affect the processing.
We'll see how the system breaks down the text into smaller pieces.
Smaller chunks might capture more specific information but lose context.
Larger chunks maintain more context but might include irrelevant information.
Finding the right balance is crucial for effective retrieval.
"""

# Test different chunk sizes
chunk_sizes = [100, 200, 300, 400, 500]
results = test_chunk_sizes(test_text, chunk_sizes)

print("Chunk Size Analysis:")
for size, data in results.items():
    print(f"\nChunk Size: {size}")
    print(f"Number of chunks: {data['num_chunks']}")
    print(f"Average chunk size: {data['avg_chunk_size']:.2f}")

## 3. Customizing the Prompt Template

Let's explore how different prompt templates affect the quality of responses.

In [ ]:
from rag_system import RAGSystem
from langchain.prompts import PromptTemplate

# Different prompt templates
templates = {
    "basic": """Use the following context to answer the question:
Context: {context}
Question: {question}
Answer:""",
    
    "detailed": """You are a helpful assistant. Use the following context to provide a detailed answer to the question.
If you don't know the answer, say so. Don't make up information.

Context: {context}

Question: {question}

Please provide a comprehensive answer:""",
    
    "technical": """As a technical expert, analyze the following context and provide a precise answer to the question.
Focus on technical accuracy and clarity.

Context: {context}
Question: {question}

Technical Answer:"""
}

# Test different prompts
def test_prompts(question, context):
    results = {}
    for name, template in templates.items():
        prompt = PromptTemplate(template=template, input_variables=["context", "question"])
        rag = RAGSystem()
        rag.qa_chain.chain_type_kwargs["prompt"] = prompt
        response = rag.query(question)
        results[name] = response["answer"]
    return results

# Example test
test_context = "The system requires Python 3.8 or higher, 8GB RAM, and 20GB storage."
test_question = "What are the system requirements?"

results = test_prompts(test_question, test_context)
for name, answer in results.items():
    print(f"\n{name.title()} Prompt:")
    print(answer)

## 4. Adding UI Features

Let's explore how to add new features to the Streamlit interface.

In [ ]:
import streamlit as st

# Example of new UI features
def enhanced_ui():
    st.title("Enhanced RAG System Interface")
    
    # Document analysis section
    st.header("Document Analysis")
    uploaded_file = st.file_uploader("Upload a document", type=["txt", "pdf"])
    if uploaded_file:
        # Show document statistics
        st.subheader("Document Statistics")
        col1, col2 = st.columns(2)
        with col1:
            st.metric("File Size", f"{len(uploaded_file.getvalue()) / 1024:.2f} KB")
        with col2:
            st.metric("File Type", uploaded_file.type)
    
    # Advanced search options
    st.header("Advanced Search")
    search_type = st.radio(
        "Search Type",
        ["Semantic Search", "Keyword Search", "Hybrid Search"]
    )
    
    # Results visualization
    st.header("Results Visualization")
    visualization_type = st.selectbox(
        "Visualization Type",
        ["Text", "Graph", "Timeline"]
    )
    
    # Example of how to use these features
    st.code("""
# In your app.py, you can add these features like this:
if st.sidebar.checkbox('Show Advanced Options'):
    enhanced_ui()
    """, language="python")

## 5. Exploring Different Language Models

Let's see how different language models affect the quality of responses.

In [ ]:
from langchain_community.llms import Ollama

def compare_models(question, context):
    models = {
        "llama2": "llama2:1.3b",
        "mistral": "mistral",
        "codellama": "codellama"
    }
    
    results = {}
    for name, model in models.items():
        try:
            llm = Ollama(model=model)
            prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
            response = llm(prompt)
            results[name] = response
        except Exception as e:
            results[name] = f"Error: {str(e)}"
    
    return results

# Example test
test_context = "The system requires Python 3.8 or higher, 8GB RAM, and 20GB storage."
test_question = "What are the system requirements?"

results = compare_models(test_question, test_context)
for model, answer in results.items():
    print(f"\n{model.title()} Model:")
    print(answer)

## 6. Best Practices and Tips

Here are some additional tips for working with the RAG system:

1. **Document Preprocessing**
   - Clean and normalize text before processing
   - Remove unnecessary formatting
   - Handle special characters appropriately

2. **Chunking Strategies**
   - Consider document structure when chunking
   - Use overlapping chunks to maintain context
   - Adjust chunk size based on content type

3. **Prompt Engineering**
   - Be specific in your prompts
   - Include examples when possible
   - Test different prompt variations

4. **Performance Optimization**
   - Cache embeddings when possible
   - Use batch processing for large documents
   - Monitor memory usage

5. **Error Handling**
   - Implement proper error handling
   - Log errors for debugging
   - Provide user-friendly error messages

## 7. Advanced Features to Explore

Here are some advanced features you might want to implement:

1. **Document Versioning**
   - Track changes in documents
   - Maintain version history
   - Compare different versions

2. **Multi-Modal Support**
   - Handle images and text together
   - Extract text from images
   - Generate image descriptions

3. **Custom Embeddings**
   - Train custom embedding models
   - Fine-tune for specific domains
   - Optimize for particular use cases

4. **Advanced Retrieval**
   - Implement hybrid search
   - Add filtering capabilities
   - Support complex queries

5. **Analytics and Monitoring**
   - Track system performance
   - Monitor user interactions
   - Generate usage reports